# Read property data

In [1]:
# 这里爬的是所有房子的数据，下面路径改一下
import pandas as pd
import re
property_data_2022 = pd.read_json('../data/raw/2022_house_data.json')

In [2]:
property_data_2022.head()

,https://www.domain.com.au/101-25-33-wills-street-melbourne-vic-3000-14868152,https://www.domain.com.au/2905b-11-rose-lane-melbourne-vic-3000-16074279,https://www.domain.com.au/1901-368-st-kilda-road-melbourne-vic-3000-16073890,https://www.domain.com.au/1211-200-spencer-street-melbourne-vic-3000-16073870,https://www.domain.com.au/1008-380-little-lonsdale-street-melbourne-vic-3000-16072582,https://www.domain.com.au/3-27-flinders-lane-melbourne-vic-3000-16072122,https://www.domain.com.au/611-408-lonsdale-street-melbourne-vic-3000-16071839,https://www.domain.com.au/102-9-bligh-place-melbourne-vic-3000-16071806,https://www.domain.com.au/1715-220-spencer-street-melbourne-vic-3000-16071670,https://www.domain.com.au/104-300-swanston-street-melbourne-vic-3000-13671355,...,https://www.domain.com.au/6-2-4-pymble-avenue-inverloch-vic-3996-15660207,https://www.domain.com.au/6-bruce-street-inverloch-vic-3996-16071116,https://www.domain.com.au/16-kenneth-street-inverloch-vic-3996-15315976,https://www.domain.com.au/25a-veronica-street-inverloch-vic-3996-16006402,https://www.domain.com.au/16b-sandy-mount-avenue-inverloch-vic-3996-16006035,https://www.domain.com.au/28-beachcomber-drive-inverloch-vic-3996-15920343,https://www.domain.com.au/14-inverloch-parade-inverloch-vic-3996-16040296,https://www.domain.com.au/10-hopetoun-street-inverloch-vic-3996-15935242,https://www.domain.com.au/inverloch-vic-3996-14434367,https://www.domain.com.au/64-dixon-street-inverloch-vic-3996-15951739
name,101/25-33 Wills Street Melbourne VIC 3000,2905B/11 Rose Lane Melbourne VIC 3000,1901/368 St Kilda Road Melbourne VIC 3000,1211/200 Spencer Street Melbourne VIC 3000,1008/380 Little Lonsdale Street Melbourne VIC ...,3/27 Flinders Lane Melbourne VIC 3000,611/408 Lonsdale Street Melbourne VIC 3000,102/9 Bligh Place Melbourne VIC 3000,1715/220 Spencer Street Melbourne VIC 3000,104/300 Swanston Street Melbourne VIC 3000,...,6/2-4 Pymble Avenue Inverloch VIC 3996,6 Bruce Street Inverloch VIC 3996,16 Kenneth street Inverloch VIC 3996,25A Veronica Street Inverloch VIC 3996,16B Sandy Mount Avenue Inverloch VIC 3996,28 Beachcomber Drive Inverloch VIC 3996,14 Inverloch Parade Inverloch VIC 3996,10 Hopetoun Street Inverloch VIC 3996,Inverloch VIC 3996,64 Dixon Street Inverloch VIC 3996
cost_text,$400,$575 per week,$1800 Per Week,$480 per week,$400 pw,$420 per week,$350,$600,$600,$550,...,$420 wk,$400,$470 wk,400 per week,$400.00 per week,$550 wk,$420 wk,$370 wk,$475 wk,Contact Agent
property_type,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,...,Apartment / Unit / Flat,House,House,House,House,House,House,House,House,House
property_feature,"[Friday, 7th October 2022, $1738]","[Tuesday, 20th September 2022, $2499]","[Available Now, $10800]","[Tuesday, 18th October 2022, $2085]","[Friday, 9th September 2022, $1738]","[Wednesday, 28th September 2022, $1825]",[Available Now],"[Available Now, $2608]","[Monday, 12th September 2022, $2607]","[Available Now, $2390]",...,"[Available Now, $1825]","[Monday, 19th September 2022, $1733, 625.7m²]","[Wednesday, 14th September 2022, $2050]","[Available Now, $1738]","[Available Now, $1738]","[Available Now, $2390]","[Available Now, $1820]","[Available Now, $1610]","[Available Now, $2065]","[Available Now, 200m²]"
coordinates,"[-37.8107757, 144.9569269]","[-37.815446, 144.9541165]","[-37.8318086, 144.9710272]","[-37.816228, 144.9532465]","[-37.8121026, 144.9581291]","[-37.8147317, 144.9733655]","[-37.8125979, 144.9604012]","[-37.817317, 144.96371]","[-37.8157157, 144.9529559]","[-37.8106088, 144.9645798]",...,"[-38.6332252, 145.7367946]","[-38.6303499, 145.7311203]","[-38.629687, 145.7322922]","[-38.6428993, 145.7101579]","[-38.6340011, 145.725239]","[-38.6412502, 145.7031282]","[-38.6287143, 145.7358082]","[-38.6341243, 145.7306862]","[-38.6314613, 145.

In [3]:
property_data_2022 = property_data_2022.T
property_data_2022 = property_data_2022.reset_index(drop=True)

In [4]:
property_data_2022.head()

,name,cost_text,property_type,property_feature,coordinates,rooms,desc
0,101/25-33 Wills Street Melbourne VIC 3000,$400,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur
1,2905B/11 Rose Lane Melbourne VIC 3000,$575 per week,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell
2,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin
3,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing
4,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent


# Get postcode from address

In [5]:
def extract_postcode (row):
    split_word = 'VIC'
    postcode = row['name'].partition(split_word)[2]
    postcode = postcode.strip()
    return postcode

In [6]:
# Checking the postcode
property_data_2022.apply (lambda row: extract_postcode(row), axis=1)

0        3000
1        3000
2        3000
3        3000
4        3000
         ... 
15087    3996
15088    3996
15089    3996
15090    3996
15091    3996
Length: 15092, dtype: object

In [7]:
# Assigning the postcode
property_data_2022["postcode"] = property_data_2022.apply (lambda row: extract_postcode(row), axis=1)

# Scrapping the SA2 name matches to postcodes
# 爬和邮编对应的 SA2 name

In [8]:
# getting the the table postcodes matches to SA2 region name
# The data scrape from https://www.matthewproctor.com/full_australian_postcodes_vic
import requests
from bs4 import BeautifulSoup
url = "https://www.matthewproctor.com/full_australian_postcodes_vic" #网址
data = requests.get(url).text  #获取数据
soup = BeautifulSoup(data, 'html.parser') #获取数据中想要的东西

In [9]:
table = soup.find("table", {"class": "table table-striped"}) #获取其中的表格

In [10]:
df = pd.DataFrame(columns=['postcodes', 'SA2_names'])

# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[1].text.strip()
        SA2_name = columns[19].text.strip()
        df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_48983/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_48983/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_48983/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_48983/3783446249.py:11: FutureWarning: The f

In [11]:
sa2_dataframe = df.iloc[1:]

In [12]:
# checking the results we scrapped
sa2_dataframe.head()

,postcodes,SA2_names
1,3000,Melbourne
2,3001,Melbourne
3,3002,East Melbourne
4,3003,West Melbourne
5,3004,Southbank


# Adding the SA2 name to each postcode

In [13]:
def add_sa2(row,sa2_dataframe):
    postcode = row['postcode']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    return sa2

In [14]:
property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

0                            [Melbourne]
1                            [Melbourne]
2                            [Melbourne]
3                            [Melbourne]
4                            [Melbourne]
                      ...               
15087    [Wonthaggi - Inverloch, Foster]
15088    [Wonthaggi - Inverloch, Foster]
15089    [Wonthaggi - Inverloch, Foster]
15090    [Wonthaggi - Inverloch, Foster]
15091    [Wonthaggi - Inverloch, Foster]
Length: 15092, dtype: object

In [15]:
property_data_2022["sa2_region"] = property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

# Starting the data preprocssing:(2022 data)
1. extract the cost out,remove all the data which does not have a valid rent cost,找到所有per week或者pw前面的数字
2. seperate the number of bedroom, number bathroom and number of parking into an individual new column （✅）
3. Adding the SA2 population 2022
4. Adding the SA2 Income 2022
5. Adding the distance to the hospital.
6. Adding the distance to school.
7. 创建新的一列 apartment/unit/flat和house然后用0,1表示. (如果需要的话)


In [16]:
def data_preprocessing_22(data_set):
    #extract the price per week out
    #现在问题是有很多数据的价格都有无用的信息
    all_rent = []
    bed_num=[]
    bath_num=[]
    park_num=[]
    for value in data_set['cost_text']:
        pattern = re.compile(r'\d+\.?\d*')
        all_rent.append(pattern.findall(value))
    data_set['cost_text'] = all_rent
    #seperate the bedroom, bathroom and parking area
    for value in data_set['rooms']:
        bed_num.append(re.findall(r'(\d+) Bed', str(value))) #找到bed前面的数字
        bath_num.append(re.findall(r'(\d+) Bath', str(value)))
        park_num.append(re.findall(r'(\d+) Parking', str(value)))
    data_set['bedroom_num'] = bed_num
    data_set['bathroom_num'] = bath_num
    data_set['parking_num'] = park_num
    return data_set
property_data_2022=data_preprocessing_22(property_data_2022) 

In [17]:
property_data_2022.head()

,name,cost_text,property_type,property_feature,coordinates,rooms,desc,postcode,sa2_region,bedroom_num,bathroom_num,parking_num
0,101/25-33 Wills Street Melbourne VIC 3000,[400],Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur,3000,[Melbourne],[1],[1],[]
1,2905B/11 Rose Lane Melbourne VIC 3000,[575],Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell,3000,[Melbourne],[2],[1],[1]
2,1901/368 St Kilda Road Melbourne VIC 3000,[1800],Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin,3000,[Melbourne],[3],[2],[2]
3,1211/200 Spencer Street Melbourne VIC 3000,[480],Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing,3000,[Melbourne],[1],[1],[1]
4,1008/380 Little Lonsdale Street Melbourne VIC ...,[400],Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent,3000,[Melbourne],[1],[1],[]


In [18]:
property_data_2022.to_csv('../data/raw/2022_house_data.csv')